In [1]:
import numpy as np 
import pandas as pd 
import re
import nltk 
import matplotlib.pyplot as pl
import pickle
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

In [2]:
#Read the training data into a data frame 
reviews = pd.read_json("data_train.json")

,stars,useful,funny,cool,text,date
0,1,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36
1,5,0,0,0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33
2,5,3,0,0,I have to say that this office really has it t...,2016-11-09 20:09:03
3,5,0,0,0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38
4,1,7,0,0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38


In [3]:
# Holds all the features/ text attribute within each review
features = reviews.iloc[:, 4].values

# Holds the stars for each review
labels = reviews.iloc[:, 0].values

In [4]:
#An array that holds every review once it is processed
processed_reviews = []

In [5]:
for sentence in range(0, len(features)):
    # Remove all the special characters
    processed_text = re.sub(r'\W', ' ', str(features[sentence]))

    # remove all single characters
    processed_text= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_text)

    # Substituting multiple spaces with single space
    processed_text = re.sub(r'\s+', ' ', processed_text, flags=re.I)

    # Converting to Lowercase
    processed_text = processed_text.lower()

    #Add the processed review to the array of processed reviews
    processed_reviews.append(processed_text)

In [6]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
# Tfidf vectorizert with the 2500 most common unique words
# Only counts words that appear in at least 7 reviews
# Only counts words that appear in less than 80% of reviews
# Remove all english stop words
vectorizer = TfidfVectorizer (max_features=2500, min_df=7, max_df=0.8, stop_words=stopwords.words('english'))
processed_reviews = vectorizer.fit_transform(processed_reviews).toarray()

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(processed_features, labels, test_size=0.2, random_state=0)

In [10]:
from sklearn.linear_model import LogisticRegression

In [11]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
# Save to file in the current working directory
pkl_filename = "nGramsLogRegression.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)

# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)

In [19]:
predictions = pickle_model.predict(X_train)

In [20]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

#print(confusion_matrix(y_train,predictions))
#print(classification_report(y_train,predictions))
#print(accuracy_score(y_train, predictions))

In [21]:
print(confusion_matrix(y_train,predictions))
print(classification_report(y_train,predictions))
print(accuracy_score(y_train, predictions))

[[ 33412   2073    951    735   2117]
 [  7729   5479   4289   2282   1774]
 [  2554   2105  11082  10042   3793]
 [   947    435   3512  28396  25572]
 [  1194    145    610  10760 105448]]
              precision    recall  f1-score   support

           1       0.73      0.85      0.79     39288
           2       0.54      0.25      0.34     21553
           3       0.54      0.37      0.44     29576
           4       0.54      0.48      0.51     58862
           5       0.76      0.89      0.82    118157

    accuracy                           0.69    267436
   macro avg       0.62      0.57      0.58    267436
weighted avg       0.67      0.69      0.67    267436

0.6873308006401532
